# Model Deployment on Vertex AI with Scikit-Learn

Contributors: michaelmenzel@google.com

## Local Model Training

In [13]:
from google.cloud import storage

import numpy as np

from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

import joblib
import os

In [14]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.1, random_state=2184)
clf = svm.SVC(gamma=0.001)
clf.fit(X_train, y_train)

SVC(gamma=0.001)

In [16]:
predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=iris.target_names))

              precision    recall  f1-score   support

      setosa       0.43      1.00      0.60         3
  versicolor       0.50      0.33      0.40         6
   virginica       1.00      0.67      0.80         6

    accuracy                           0.60        15
   macro avg       0.64      0.67      0.60        15
weighted avg       0.69      0.60      0.60        15



In [17]:
model_file = 'model.joblib'
gcs_path = 'gs://sandbox-michael-menzel-model/sklearn-iris-classifier'

joblib.dump(clf, model_file)

blob = storage.blob.Blob.from_string(os.path.join(gcs_path, model_file), client=storage.Client())
blob.upload_from_filename('model.joblib')

## Model Deployment

In [18]:
!pip install -q --user --no-warn-conflicts google-cloud-aiplatform

In [19]:
from google.cloud import aiplatform

aiplatform.init(location='europe-west4')

In [20]:
vertex_ai_model = aiplatform.Model.upload(
    display_name='sklearn-iris-classifier',
    artifact_uri=gcs_path,
    serving_container_image_uri='europe-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest',
)
 
vertex_ai_model.wait()

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/928871478446/locations/europe-west4/models/5176060938918297600/operations/6912226083060318208
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/928871478446/locations/europe-west4/models/5176060938918297600
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/928871478446/locations/europe-west4/models/5176060938918297600')


In [21]:
vertex_ai_endpoint = vertex_ai_model.deploy(
    deployed_model_display_name='sklearn-iris-classifier-model',
    traffic_split={"0": 100},
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=1,
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272/operations/5633203788887097344
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/928871478446/locations/europe-west4/endpoints/3724564852765622272')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/928871478446/locations/europe-west4/endpoints/3724564852765622272
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272/operations/3327360779673403392
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/928871

In [22]:
vertex_ai_predicted = vertex_ai_endpoint.predict(X_test.tolist())
print(metrics.classification_report(y_test, vertex_ai_predicted.predictions, target_names=iris.target_names))

              precision    recall  f1-score   support

      setosa       0.43      1.00      0.60         3
  versicolor       0.50      0.33      0.40         6
   virginica       1.00      0.67      0.80         6

    accuracy                           0.60        15
   macro avg       0.64      0.67      0.60        15
weighted avg       0.69      0.60      0.60        15



In [23]:
vertex_ai_predicted

Prediction(predictions=[0.0, 2.0, 1.0, 0.0, 1.0, 2.0, 0.0, 1.0, 0.0, 2.0, 1.0, 0.0, 0.0, 2.0, 0.0], deployed_model_id='3740714479554396160', explanations=None)

## Cleanup

In [24]:
vertex_ai_endpoint.delete(force=True)

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272/operations/1541683527421001728
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/928871478446/locations/europe-west4/endpoints/3724564852765622272
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/928871478446/locations/europe-west4/operations/6673535302809681920
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/928871478446/locations/europe-west4/endpoints/3724564852765622272


In [25]:
vertex_ai_model.delete()

INFO:google.cloud.aiplatform.base:Deleting Model : projects/928871478446/locations/europe-west4/models/5176060938918297600
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/928871478446/locations/europe-west4/operations/3635857354148282368
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/928871478446/locations/europe-west4/models/5176060938918297600
